In [2]:
##Import relevant packages:
import requests
import pandas as pd
import numpy as np
from time import sleep

In [3]:
#API key for Riot API
riot_api_key = "RGAPI-2328d769-4f75-42c0-8673-8001bed0ee54"

#example request url for player named 'Samikin' 
request_url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Samikin"
request_url_api = request_url + "?api_key=" + riot_api_key

In [4]:
"""
I will use the same pipeline as the non-text data retrieval from the Riot API but this time I will focus on text data.

Below is a copy of what I used to retrieve match data (record data)
"""


#looping through to request data on a player's recent games
#only looking at player data from NA in each divison at tier 2
#(ie. bronze 2, gold 2, etc.)

#function get all summoner_names in a certain division
#like GOLD, DIAMOND, PLATINUM
#param: division as string, riot API key
#return: puuid as string
def get_league(division, riot_api_key):
    request_url = ("https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/" + division + 
                   "/II?page=1" + "&api_key=" + riot_api_key) #create correct request url
    resp = requests.get(request_url) #get response after requesting from url
    players = resp.json() #get dict of page 1 of players in certain division
    league_list = [] #init empty list to append player summoner names
    for dict in players: #loop thru each player in the dict
        player = dict['summonerName'] #add the summoner name to empty list
        league_list.append(player)
    return league_list


#function get puuid from summoner name
#param: player name as string, riot API key
#return: puuid as string
def get_puuid(summoner_name, riot_api_key): 
    request_url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + summoner_name
    request_url_api = request_url + "?api_key=" + riot_api_key #create request url
    resp = requests.get(request_url_api) #get response
    if resp.status_code == 200:
        player_info = resp.json() #parse player info
        if 'puuid' in player_info: #only looking for dictionary where 'puuid' exists
            puuid = player_info['puuid'] #retrieve only puuid to use later
    return puuid


#function to get 10 match ID's with puuid
#param: puuid as string, riot API key
#return: match id's as list of strings 
def get_matches(puuid, riot_api_key): 
    match_url = (
        "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/" +
        puuid +
        "/ids?start=0&count=10" +
        "&api_key=" +
        riot_api_key
    ) #create request url
    resp = requests.get(match_url) #request with url
    matches = resp.json() #parse it
    return matches #we get the 10 most recent matches of a player


In [8]:
"""
I will use mostly the same pipeline from the non-text data retrieval with modifications for just the position of each player

Below is a similar function to retrieve text data
"""

#function to get specific types of data from ranked match data
#param: puuid as string, match_data as dict
#return: position text data, champion (fictional character in the game) text data 
##will utilize position to create prediction model of which champions plays in which position
##columns = ['position','champion']
def player_data(puuid, match_data):
    match = np.nan
    if 'info' in match_data: #only looking for the data where 'info' dictionary exists
        if match_data['info']['queueId'] == 420: #only ranked matches are wanted
            player_index = match_data['metadata']['participants'].index(puuid) #get index of player in the match
            #because the dictionary is made according to the index of the player's puuid in the initial dict
            champion = match_data['info']['participants'][player_index]['championName'] #the champion played
            position = match_data['info']['participants'][player_index]['individualPosition'] #number of kills in the match

            match = pd.Series([champion, position],
                          index=['champion','position'])
    return match


#function to get match DATA with a list of match id
#param: puuid as string, match ID's as list of strings, riot API key, empty df
#empty df should have columns with these names in this order:
#['champion','position']
#return: updated df with data
def get_match_data(puuid, matches, riot_api_key, empty_df):
    for match in matches:
        match_data_url = (
            "https://americas.api.riotgames.com/lol/match/v5/matches/" +
            match + 
            "?api_key=" +
            riot_api_key
        ) #create request url
        match_data = requests.get(match_data_url) #request the url
        match_data_parse = match_data.json() #parse the response
        data_series = player_data(puuid, match_data_parse) #get all desired match variables/data
        data_series2 = pd.DataFrame([data_series]) #change series to df
        empty_df = pd.concat([empty_df, data_series2], ignore_index=True) #concat to empty df to add to the dataset

    return empty_df

In [9]:
"""
Test code for player named "Samikin"
"""

########################TESTING FUNCTIONS###########################
#Using player named 'Samikin' as an example to test the functions
example = 'Samikin' #an example player
empty_df_example = pd.DataFrame(columns=['champion','position']) #init df of variables
puuid_example = get_puuid(example, riot_api_key) #getting puuid of player
matches_example = get_matches(puuid_example, riot_api_key) #get recent 10 matches with puuid
text_data_example = get_match_data(puuid_example, matches_example, riot_api_key, empty_df_example) #get desired match data in each match
#will also concat to df as a row (match = row)

In [10]:
"""
Create actual dataset of different ranks in ranked solo/duo 5v5 on summoner's rift (TEXT DATA).
I will ignore the top 300 ranked 'Challenger' players for the text data.
Same process as record data retrieval but this time only for position and champion played.
"""

#1. use get_league() function to get 205 summoner names in a given division
#2. loop over the list returned by get_league() to run through each player
#3. get puuid of each player
#4. get recent 10 matches of each player
#5. get text data if the match is ranked solo/duo 5v5 on summoner's rift
##### ranked solo/duo 5v5 on summoner's rift is the typical queue type that player use to determine their skill at the game
##### this is domain knowledge but it is the dataset I will be working with in this project


#init empty df with desired variables for IRON
df_iron = pd.DataFrame(columns=['champion','position'])

#create list of all summoner names in division iron tier 2 (IRON II)
league_list = get_league('IRON', riot_api_key)
#run through the necessary functions to update df with desired match data
for summoner in league_list:
    puuid_summoner = get_puuid(summoner, riot_api_key) #getting puuid of player
    matches_summoner = get_matches(puuid_summoner, riot_api_key) #get recent 10 matches with puuid
    df_iron = get_match_data(puuid_summoner, matches_summoner, riot_api_key, df_iron) #update df with match data
#to categorize this data of Iron II players, I will manually add a column with their respective rank
df_iron['rank'] = 'iron'

In [12]:
#Now do the same for each respective division
#Divisions: Iron, Bronze, Silver, Gold, Platinum, Emerald, Diamond
#Do same operations for all ranks aside from Iron...

#init empty df with desired variables for BRONZE
df_bronze = pd.DataFrame(columns=['champion','position'])

#create list of all summoner names in division bronze tier 2
league_list = get_league('BRONZE', riot_api_key)
#run through the necessary functions to update df with desired match data
for summoner in league_list:
    try:
        puuid_summoner = get_puuid(summoner, riot_api_key) #getting puuid of player
        matches_summoner = get_matches(puuid_summoner, riot_api_key) #get recent 10 matches with puuid
        df_bronze = get_match_data(puuid_summoner, matches_summoner, riot_api_key, df_bronze) #update df with match data
    except Exception as e:
        print(e)
#add rank to new col
df_bronze['rank'] = 'bronze'

cannot access local variable 'puuid' where it is not associated with a value
cannot access local variable 'puuid' where it is not associated with a value
cannot access local variable 'puuid' where it is not associated with a value
cannot access local variable 'puuid' where it is not associated with a value
cannot access local variable 'puuid' where it is not associated with a value
cannot access local variable 'puuid' where it is not associated with a value
cannot access local variable 'puuid' where it is not associated with a value
cannot access local variable 'puuid' where it is not associated with a value


In [13]:
#init empty df with desired variables for SILVER
df_silver = pd.DataFrame(columns=['champion','position'])

#create list of all summoner names in division silver tier 2 
league_list = get_league('SILVER', riot_api_key)
#run through the necessary functions to update df with desired match data
for summoner in league_list:
    try:
        puuid_summoner = get_puuid(summoner, riot_api_key) #getting puuid of player
        matches_summoner = get_matches(puuid_summoner, riot_api_key) #get recent 10 matches with puuid
        df_silver = get_match_data(puuid_summoner, matches_summoner, riot_api_key, df_silver) #update df with match data
    except Exception as e:
        print(e)
#add rank to new col
df_silver['rank'] = 'silver'

In [14]:
#init empty df with desired variables for GOLD
df_gold = pd.DataFrame(columns=['champion','position'])

#create list of all summoner names in division gold tier 2 
league_list = get_league('GOLD', riot_api_key)
#run through the necessary functions to update df with desired match data
for summoner in league_list:
    sleep(0.5)
    try:
        puuid_summoner = get_puuid(summoner, riot_api_key) #getting puuid of player
        matches_summoner = get_matches(puuid_summoner, riot_api_key) #get recent 10 matches with puuid
        df_gold = get_match_data(puuid_summoner, matches_summoner, riot_api_key, df_gold) #update df with match data
    except Exception as e:
        print(e)
#add rank to new col
df_gold['rank'] = 'gold'

cannot access local variable 'puuid' where it is not associated with a value
cannot access local variable 'puuid' where it is not associated with a value
cannot access local variable 'puuid' where it is not associated with a value


In [15]:

#init empty df with desired variables for PLATINUM
df_platinum = pd.DataFrame(columns=['champion','position'])

#create list of all summoner names in division platinum tier 2 
league_list = get_league('PLATINUM', riot_api_key)
#run through the necessary functions to update df with desired match data
for summoner in league_list:
    sleep(0.5)
    try:
        puuid_summoner = get_puuid(summoner, riot_api_key) #getting puuid of player
        matches_summoner = get_matches(puuid_summoner, riot_api_key) #get recent 10 matches with puuid
        df_platinum = get_match_data(puuid_summoner, matches_summoner, riot_api_key, df_platinum) #update df with match data
    except Exception as e:
        print(e)
#add rank to new col
df_platinum['rank'] = 'platinum'

cannot access local variable 'puuid' where it is not associated with a value


In [16]:
#init empty df with desired variables for EMERALD
df_emerald = pd.DataFrame(columns=['champion','position'])

#create list of all summoner names in division emerald tier 2 
league_list = get_league('EMERALD', riot_api_key)
#run through the necessary functions to update df with desired match data
for summoner in league_list:
    try:
        puuid_summoner = get_puuid(summoner, riot_api_key) #getting puuid of player
        matches_summoner = get_matches(puuid_summoner, riot_api_key) #get recent 10 matches with puuid
        df_emerald = get_match_data(puuid_summoner, matches_summoner, riot_api_key, df_emerald) #update df with match data
    except Exception as e:
        print(e)
#add rank to new col
df_emerald['rank'] = 'emerald'

cannot access local variable 'puuid' where it is not associated with a value


In [17]:
#init empty df with desired variables for DIAMOND
df_diamond = pd.DataFrame(columns=['champion','position'])

#create list of all summoner names in division diamond tier 2 
league_list = get_league('DIAMOND', riot_api_key)
#run through the necessary functions to update df with desired match data
for summoner in league_list:
    try:
        puuid_summoner = get_puuid(summoner, riot_api_key) #getting puuid of player
        matches_summoner = get_matches(puuid_summoner, riot_api_key) #get recent 10 matches with puuid
        df_diamond = get_match_data(puuid_summoner, matches_summoner, riot_api_key, df_diamond) #update df with match data
    except Exception as e:
        print(e)
#add rank to new col
df_diamond['rank'] = 'diamond'

cannot access local variable 'puuid' where it is not associated with a value


In [19]:
"""
write all dataframes in .csv files. This is for TEXT DATA.
"""
#write the created df_LEAGUE into a .csv file called 'league_LEAGUE_data.csv'
#for league = iron, bronze, silver, gold, platinum, emerald, diamond
df_iron.to_csv('iron_text_data.csv', index = False)
df_bronze.to_csv('bronze_text_data.csv', index = False)
df_silver.to_csv('silver_text_data.csv', index = False)
df_gold.to_csv('gold_text_data.csv', index = False)
df_platinum.to_csv('platinum_text_data.csv', index = False)
df_emerald.to_csv('emerald_text_data.csv', index = False)
df_diamond.to_csv('diamond_text_data.csv', index = False)